In [ ]:
# -*- coding: utf-8 -*-

import pymongo
from pymongo import MongoClient
import pandas as pd

In [ ]:
df_setores = pd.read_excel('C:/Users/CaioEstrella/Desktop/Projetos_Scraping/TeleListas/setores.xlsx', encoding='utf-8')
setores = list(df_setores['setor'])

# Adiciona elementos ao 'setores'

In [ ]:
## mais palavras-chave
add = ['mercadinho', 'frutas legumes e verduras', 'Verduras e Legumes', 'quitandas']
for i in add:
    setores.append(i)

In [ ]:
cluster = MongoClient("mongodb+srv://CaioEstrella:<my_password_add_later>@clustertelelistas.dptye.mongodb.net/TeleListas?retryWrites=true&w=majority")
db = cluster["TeleListas"]
collection = db["Tabela"]

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from selenium import webdriver
import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
chrome_options = Options()
chrome_options.add_argument("--headless")

p = str(Path(os.getcwd()))
driver = webdriver.Chrome(p+'/chromedriver.exe', options=chrome_options)
wait = WebDriverWait(driver, 30)

In [ ]:
#Informações para fingir ser um navegador
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
nome_empresas = []
telefones2 = []
endereco = []
bairro = []
cidade = []
estado = []
descricao = []
servicos=[]

from unidecode import unidecode

for setor in setores[:]:
    
    setor = unidecode(setor)
    set_ = setor.replace(' ', '+')
      
    url = "https://www.telelistas.net/rj/rio+de+janeiro/"+set_+"?pag=1"
    driver.get(url)
     
    try:
        contagem = driver.find_element_by_xpath("//div[contains(text(), 'Pág.')]")
        print('contagem.text: ',contagem.text)
        contagem = int(contagem.text.split("/")[1]) 
    except NoSuchElementException:
        contagem = 2
    
    #print(contagem)
    for pag in range(0, contagem): #em cada página
        
        print("current_url: ",driver.current_url)
        html = Request(driver.current_url, headers=header)
        html = urlopen(html).read()
        
        bs = BeautifulSoup(html, 'html.parser')
       
        ## Nome das empresas
        empresas = bs.find_all("h3", {"class": "no-h3 cursor-pointer wrap"})

        for empresa in empresas:
            nome_empresas.append(empresa.text)
            print(empresa.text)
        

        ## telefones2
        lista_info = driver.find_elements_by_xpath("//span[contains(text(),'Info')]")
        
        time.sleep(1)
        
        for i in lista_info:
              
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[0])
            i.click() #vai pra handle[1] automaticamente
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[1])
            
            time.sleep(2)          
            try:
                driver.find_element_by_class_name("btn.btn-outline-dark.btn-minfo2.mr-3.mb-2.font-weight-bold").click()
                time.sleep(0.1)
                wait.until(EC.element_to_be_clickable((By.ID, 'mostrar-telefones'))).click()
            except :
                print('erro')
                
            #driver.find_element_by_id('mostrar-telefones').click()
            #time.sleep(0.3)
            bs = BeautifulSoup(driver.page_source, 'html.parser')
            pai = bs.find("div",{"id":'mostrarnumeros'})
            #print(pai)
            para_add = []

            for child in pai.findChildren("h6" , recursive=False):
                #print(child.text)
                para_add.append(child.text.strip())

            telefones2.append(" | ".join(para_add))

            ## descricao
            try:
                el = bs.find("p", {"class":"card-text h6 text-left pb-4 nav-tabs"})
                descricao.append(el.text.strip())
            except (NoSuchElementException, AttributeError):
                descricao.append("---")

            ##serviços

            try:
                el = bs.find("ul", {"class": "h6 text-left"})
                servicos.append(el.text.strip())
            except (NoSuchElementException, AttributeError):
                servicos.append("---")


            driver.close()

            driver.switch_to.window(driver.window_handles[0])
            
        #driver.switch_to.window(driver.window_handles[0])
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        
        #########################################
        ## endereço / bairro / cidade / estado ##
        #########################################
        
        lista_div = bs.find_all("div", {"class":"q-card__section q-card__section--vert"})
        
        if setor == '_mercadinhos':
            c = 0
        else:
            c = 1
        
        for i in lista_div[c:]: ####colocar [1:]

            if not i.find("div", {"class":"caddress column"}):#len(children) == 0):
                endereco.append("NÃO CONTÉM ENDEREÇO")
                bairro.append("NÃO CONTÉM DADOS")
                cidade.append("NÃO CONTÉM DADOS")
                estado.append("NÃO CONTÉM DADOS")
                #continue
            else:    #for i in children:
                end = i.find("div", {"class":"caddress column"}).find_all("div")[0] #A PRIMEIRA DIV
                end = end.text.replace('room', '')
                #print("entrando no endereço: ", end.replace("\n",""))
                endereco.append(end.replace("\n","").strip())

                end2 = i.find("div", {"class":"caddress column"}).find_all("div")[1] # A SEGUNDA DIV

                #print(end2.text)

                separado = end2.text.split(" - ")
                #print(separado)
                if(len(separado) == 3):
                    bairro.append(separado[0].replace("\n","").strip())
                    cidade.append(separado[1].replace("\n","").strip())
                    estado.append(separado[2].replace("\n","").strip())
                elif(len(separado) == 2):
                    bairro.append('--')
                    cidade.append(separado[0].replace("\n","").strip())
                    estado.append(separado[1].replace("\n","").strip())
                else:
                    bairro.append("--")
                    cidade.append("--")
                    estado.append("--")
                    
        driver.switch_to.window(driver.window_handles[0])
    
        driver.find_element(By.XPATH, '//div[text()="próximo"]').click()
        time.sleep(0.5)
        print("clicou")
        
        #print("setor_serie: ",len(setor_serie))    
        print("nome_empresas: ",len(nome_empresas))
        print("telefones: ",len(telefones2))
        print('endereco: ',len(endereco))
        print('bairro: ',len(bairro))
        print('cidade: ',len(cidade))
        print('estado: ',len(estado))
        print('descrição: ',len(descricao))
        print('serviços e produtos: ',len(servicos))
          
        
    setor_serie = [setor] * len(nome_empresas)   
    
    print('###### FIM #######')
    
    print("nome_empresas: ",len(nome_empresas))
    print("telefones: ",len(telefones2))
    print('endereco: ',len(endereco))
    print('bairro: ',len(bairro))
    print('cidade: ',len(cidade))
    print('estado: ',len(estado))
    print('descrição: ',len(descricao))
    print('serviços e produtos: ',len(servicos))
    

##fim do loop de setores

In [ ]:
df_vai_para_db = pd.DataFrame({"Setor":setor_serie,"nome_empresas": nome_empresas, "telefone": telefones2, "endereço" : endereco,
                "bairro":bairro, "cidade":cidade , "estado": estado, "descrição":descricao, "serviços e produtos": servicos})

## adiciona uma linha por vez no banco

for index, row in df_vai_para_db.iterrows():
    collection.insert_one({
        "setor": row['Setor'],
        "nome_empresa": row['nome_empresas'],
        "telefones" : row['telefone'],
        "endereco":row['endereço'],
        "bairro":row['bairro'],
        "cidade":row['cidade'],
        "estado":row['estado'],
        "descrição":row['descrição'],
        "servicos_e_produtos":row['serviços e produtos']
    })  

In [ ]:
df_vai_para_db = df_vai_para_db.drop_duplicates()

In [ ]:
df_vai_para_db.to_excel(p+'/mercados.xlsx', index=False)